In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from bi_gru_kalman_model import KalmanBiGRUModel
from loss_chi_square import ChiSquareLoss
import pandas as pd
import numpy as np

def create_sequences(data, seq_len):
    xs, ys = [], []
    for i in range(len(data) - seq_len):
        x = data[i:i+seq_len, :-1]
        y = data[i+seq_len, -1]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

def train_model(df, input_dim, seq_len=24, hidden_dim=128, batch_size=32, epochs=100, lr=0.001):
    data = df.to_numpy()
    X, y = create_sequences(data, seq_len)
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, shuffle=False)

    X_train = torch.tensor(X_train, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.float32)
    X_val = torch.tensor(X_val, dtype=torch.float32)
    y_val = torch.tensor(y_val, dtype=torch.float32)

    train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=batch_size)

    model = KalmanBiGRUModel(input_dim=input_dim, hidden_dim=hidden_dim)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = ChiSquareLoss()

    for epoch in range(epochs):
        model.train()
        for batch_x, batch_y in train_loader:
            optimizer.zero_grad()
            preds, _ = model(batch_x)
            loss = criterion(preds, batch_y)
            loss.backward()
            optimizer.step()

    return model, val_loader
